In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from sqlalchemy import create_engine
import urllib
import datetime as dt
from pathlib3x import Path
import easygui

In [2]:

def selecionarArquivo(*pasta):
    """
    Selecionar um arquivo para servir de carga no SAP
    Returns:
        String: retorna um caminho em forma de string (texto)
    """
    global local_arquivo
    local_arquivo = ''

    if not pasta:
        arquivo = easygui.fileopenbox()
        local_arquivo = Path(arquivo)   
    else:
        print(f"o arquivo da pasta: {pasta[0]}")
        local_arquivo = pasta[0]
        
    return local_arquivo

###### Selecionar a pasta do arquivo do mês ########

localArquivo = selecionarArquivo()

In [3]:
localArquivo

WindowsPath('C:/Users/leandro.alves/Norte Energia/orcamento - General/4 - Documentos_BI/BI_Tesouraria/BASE_RECEITA_FIN/Fundos_Aplicacao/SQL_base_Aplicacao_18-09-2023.xlsx')

In [ ]:
"""
Banco de dados: 

[DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS]
[DW_SFCRI].[TESOURARIA].[TITULOS]
[DW_SFCRI].[TESOURARIA].[BANCOS]


sheets: 

TITULOS, APLICACAO_FUNDOS, BANCOS

"""


In [3]:
df_bancos = pd.read_excel(localArquivo, sheet_name='BANCOS')
df_titulos = pd.read_excel(localArquivo, sheet_name='TITULOS')
df_aplicacao = pd.read_excel(localArquivo, sheet_name='APLICACAO_FUNDOS')

In [ ]:
df_titulos['LIQUIDEZ'] = df_titulos['LIQUIDEZ'].astype(str)
df_titulos

In [ ]:
df_bancos

In [ ]:
df_aplicacao

#### carga SQL

In [5]:
conexao = pyodbc.connect('Driver={SQL Server};'
                      'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                      'Database=DW_SFCRI;'
                      'Trusted_Connection=yes;')

cursor = conexao.cursor() # Cria o cursor para manipular o banco de dados
truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS] """
# truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS_2] """
truncate_BANCOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[BANCOS] """
truncate_TITULOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[TITULOS] """

In [6]:
cursor.execute(truncate_APLICACAO_FUNDOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_BANCOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_TITULOS) # Executa a Limpeza dos dados 
cursor.commit() # Confirma

In [7]:
pst = urllib.parse.quote_plus('Driver={SQL Server};'
                                'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                                'Database=DW_SFCRI;'
                                'Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={pst}')

In [8]:
df_titulos.to_sql('TITULOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
df_bancos.to_sql('BANCOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
print('Começando a carga de aplicação')
# df_aplicacao.to_sql('APLICACAO_FUNDOS_2', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
df_aplicacao.to_sql('APLICACAO_FUNDOS', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
print('Fim da carga de aplicação')


Começando a carga de aplicaçãp
Fim da carga de aplicaçãp


#### Testes tipos

In [6]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que deseja limpar

# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(str)
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].str.strip()

In [7]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x) if x.strip() else 0.0)


In [ ]:

# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x.strip()) if isinstance(x, str) and x.strip() else 0.0)


In [ ]:
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(float)